In [ ]:
import pandas as pd
from google.colab import drive
import numpy as np
from datetime import datetime

drive.mount('/content/drive')

path = '/content/drive/MyDrive/names_EW-1996-2000.csv'

import pandas as pd
df = pd.read_csv(path)
df

Mounted at /content/drive


,rank,name,count,year,category
0,1,Jack,10779,1996,male
1,2,Daniel,10338,1996,male
2,3,Thomas,9603,1996,male
3,4,James,9385,1996,male
4,5,Joshua,7887,1996,male
...,...,...,...,...,...
4855,545,Joy,50,2000,female
4856,545,Kiah,50,2000,female
4857,545,Martina,50,2000,female
4858,545,Romy,50,2000,female


In [ ]:
import pandas as pd
import re

# Regex patterns for names
full_name_pattern = re.compile(r'^[A-Z][a-z]+\s[A-Z][a-z]+$', re.IGNORECASE)  # "John Doe"
single_name_pattern = re.compile(r'^[A-Z][a-z]+$', re.IGNORECASE)  # "John" or "Doe"

# Common non-name values (race, gender labels, etc.)
non_name_values = {"male", "female", "nonbinary", "black", "white", "asian", "hispanic", "latino", "native"}

# Function to refine name column detection
def find_name_columns(df):
    name_columns = []

    for column in df.columns:
        values = df[column].astype(str)

        match_count_full = values.apply(lambda x: bool(full_name_pattern.match(x))).sum()
        match_count_single = values.apply(lambda x: bool(single_name_pattern.match(x))).sum()
        non_name_count = values.apply(lambda x: x.lower() in non_name_values).sum()

        # Check if values are consistently short (likely a name, not descriptive text)
        avg_word_count = values.apply(lambda x: len(str(x).split())).mean()

        # Consider a column a "name column" if:
        # - At least 70% of values match name patterns
        # - Less than 10% match race/gender labels
        # - Word count average is **≤ 2** (filters out descriptive categorical data)
        if max(match_count_full, match_count_single) / len(values) > 0.7 and non_name_count / len(values) < 0.1 and avg_word_count <= 2:
            name_columns.append(column)

    return name_columns

# Identify columns containing names
name_columns = find_name_columns(df)

# Print results
if name_columns:
    print(f"✅ Name columns found: {name_columns}")
else:
    print("❌ No name columns detected.")

✅ Name columns found: ['name']


In [ ]:
import pandas as pd


# Function to extract only the first letter of names
def first_letter_only(name):
    return name[0] if isinstance(name, str) and name.strip() != "" else ""  # Ensures valid input

# Apply transformation **only to first 50 rows**
df.loc[:49, name_columns] = df.loc[:49, name_columns].applymap(first_letter_only)

# Print results
print(df.head(50))  # Show first 50 modified rows

    rank name  count  year category
0      1    J  10779  1996     male
1      2    D  10338  1996     male
2      3    T   9603  1996     male
3      4    J   9385  1996     male
4      5    J   7887  1996     male
5      6    M   7426  1996     male
6      7    R   6496  1996     male
7      8    J   6193  1996     male
8      9    S   6161  1996     male
9     10    L   5802  1996     male
10    11    J   5750  1996     male
11    12    L   5664  1996     male
12    13    C   5009  1996     male
13    14    A   4840  1996     male
14    15    B   4805  1996     male
15    16    A   4538  1996     male
16    17    H   4434  1996     male
17    18    J   4331  1996     male
18    19    G   4287  1996     male
19    20    C   4281  1996     male
20    21    W   4269  1996     male
21    22    M   4187  1996     male
22    23    O   3655  1996     male
23    24    L   3569  1996     male
24    25    C   3483  1996     male
25    26    K   2882  1996     male
26    27    N   2744  1996  

<ipython-input-3-1033100105>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df.loc[:49, name_columns] = df.loc[:49, name_columns].applymap(first_letter_only)


In [ ]:
import pandas as pd

# Function to classify individual values
def classify_name(value):
    value = str(value).strip()
    if len(value) == 1:  # Single character = Initial
        return "Initial"
    elif len(value) > 1:  # More than one character = Full Name
        return "Full Name"
    else:
        return "Unknown"


# Apply classification function to each value
for col in name_columns:
    df[f"{col} Type"] = df[col].apply(classify_name)

# Count full names and initials
for col in name_columns:
    full_names_count = (df[f"{col} Type"] == "Full Name").sum()
    initials_count = (df[f"{col} Type"] == "Initial").sum()

    # Print results
    print(f"❌ {col}: {full_names_count} full names, {initials_count} initials, Data is inconsistent. Suggest getting the full name for values with just initials.")

❌ name: 4810 full names, 50 initials, Data is inconsistent. Suggest getting the full name for values with just initials.
